In [1]:
import requests
import time
import re
import sqlite3  # 追加: データベース用ライブラリ
from bs4 import BeautifulSoup

In [2]:
base_url = "https://github.com/orgs/google/repositories"

# データの並び順を決める設定（消さないでください）
base_params = { 
    "q": "", 
    "type": "all", 
    "language": "", 
    "sort": "name" 
}


# --- データベースの準備 ---
db_name = "google_repos.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# テーブル作成
cursor.execute('''
    CREATE TABLE IF NOT EXISTS repositories (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE,
        language TEXT,
        stars INTEGER
    )
''')
conn.commit()
print(f"データベース '{db_name}' に接続しました。")

# --- メイン処理開始 ---
current_page = 1 
total_count = 0 
seen_repos = set()

print("\n" + "="*85)
print(f"| {'No.':<6} | {'リポジトリ名':<40} | {'言語':<12} | {'スター':>8} |")
print("-" * 85)

while True:
    # ここで base_params を使うため、上で定義されている必要があります
    params = base_params.copy()
    params['page'] = current_page
    print(f"\n--- Page {current_page} を取得中... ---")
    
    try:
        time.sleep(1)
        
        # ここで headers を使うため、上で定義されている必要があります
        response = requests.get(base_url, params=params, headers=headers)
        
        if response.status_code != 200:
            print(f"❌ 通信エラー: {response.status_code}")
            break
            
        soup = BeautifulSoup(response.content, 'html.parser')

        items = soup.find_all(['li', 'div'])
        valid_items_on_page = 0

        for item in items:
            # ① 名前
            name_tag = item.select_one("h3 a") or item.select_one("a[itemprop='name codeRepository']")
            if not name_tag: continue

            repo_name = name_tag.get_text(strip=True)
            # ゴミデータの除外
            if not repo_name or repo_name.isdigit() or re.match(r'^\d+(\.\d+)?k?$', repo_name):
                continue
            
            if repo_name in seen_repos:
                continue

            # ② 言語
            language = "N/A"
            # あなたが特定したクラス名
            lang_tag = item.find(class_ = "ReposListItem-module__Box_9--RH81p")
            
            # バックアップ（もしクラス名が変わっていた時のため）
            if not lang_tag:
                 lang_tag = item.select_one("span[itemprop='programmingLanguage']")

            if lang_tag:
                if lang_tag.get("itemprop") == "programmingLanguage":
                    language = lang_tag.get_text(strip=True)
                else:
                    # 親要素のテキストから抽出
                    parent_text = lang_tag.parent.get_text(" ", strip=True)
                    for trash in ["Updated", "Built", "License", "View"]:
                        if trash in parent_text:
                            parent_text = parent_text.split(trash)[0]
                    
                    cleaned = parent_text.strip()
                    parts = cleaned.split()
                    candidates = [w for w in parts if not any(c.isdigit() for c in w) and len(w) > 1]
                    if candidates:
                        language = candidates[0]

            # ③ スター数
            star_count = 0
            star_tag = item.select_one("a[href$='stargazers']")
            
            if star_tag:
                raw_star = star_tag.get_text(strip=True).replace(',', '').replace(' ', '')
                try:
                    if 'k' in raw_star:
                        star_count = int(float(raw_star.replace('k', '')) * 1000)
                    else:
                        star_count = int(raw_star)
                except:
                    pass

            # --- 画面出力 ---
            seen_repos.add(repo_name)
            total_count += 1
            valid_items_on_page += 1
            print(f"| {total_count:<6} | {repo_name:<40} | {language:<12} | {star_count:>8} |")

            # --- データベース保存 ---
            try:
                cursor.execute('''
                    INSERT OR IGNORE INTO repositories (name, language, stars)
                    VALUES (?, ?, ?)
                ''', (repo_name, language, star_count))
            except Exception as e:
                print(f"DB Error: {e}")

        conn.commit()

        if valid_items_on_page == 0:
            print(">> 有効なリポジトリが見つかりません。終了します。")
            break

        current_page += 1
        # 安全のため100ページで停止
        if current_page > 100: break

    except Exception as e:
        print(f"エラー: {e}")
        break

conn.close()
print("-" * 85)
print(f"完了: 合計 {total_count} 件をデータベースに保存しました。")

データベース 'google_repos.db' に接続しました。

| No.    | リポジトリ名                                   | 言語           |      スター |
-------------------------------------------------------------------------------------

--- Page 1 を取得中... ---
エラー: name 'headers' is not defined
-------------------------------------------------------------------------------------
完了: 合計 0 件をデータベースに保存しました。
